<a href="https://colab.research.google.com/github/whoami-Lory271/thesis-project/blob/main/LSTM_Linear_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Symbol legend

* B: batch size
* L: lookback window (aka input_size)


# Installations and imports


In [ ]:
!pip install pytorch-lightning==2.0.1.post0 --quiet
!pip install einops==0.6.1 --quiet
!pip install ipdb --quiet
!pip install wandb --quiet
# !pip install objsize --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 807.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import logging
# https://github.com/gotcha/ipdb
import ipdb
import copy
from google.colab import drive
from tqdm.notebook import tqdm
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch.nn as nn
import torch.nn.functional as F
from torch.profiler import profile, record_function, ProfilerActivity
import numpy as np
import random
from collections import defaultdict
import functools
# https://theaisummer.com/einsum-attention/
import einops
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
from sklearn.preprocessing import StandardScaler
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.profilers import PyTorchProfiler
from pytorch_lightning.tuner import Tuner

import wandb
import sys
import pickle
import time
import os
import torch.fft as fft
# import objsize

In [ ]:
drive.mount('/content/drive')
!cp /content/drive/MyDrive/Tesi/code/.netrc /root/
wandb.login()
# 024a3906e525e6d2640af94c364128bb3d33e44b

Mounted at /content/drive


wandb: Currently logged in as: desantis-1849114 (desantis). Use `wandb login --relogin` to force relogin


True

# Helper Functions

In [ ]:
# setup logger function
def setup_log(self, level):
    log = logging.getLogger(self.__class__.__name__)
    log.setLevel(level)
    return log

In [ ]:
# write log on a file
def write_log(a):
    with open("log.txt", 'w') as file:
        for row in a:
            file.write(str(row))
        log.debug("object logged")

In [ ]:
def get_patch_num(patch_len, num_var, stride):
    return (max(patch_len, num_var)-patch_len) // stride + 2

In [ ]:
# pad array with nan
def pad_nan(arr, left=0, right=0, dim=0):
    # padding the right side
    if left > 0:
        padshape = list(arr.shape)
        padshape[dim] = left
        arr = torch.cat((torch.full(padshape, np.nan).to(arr.device), arr), dim=dim)

    # padding the left side
    if right > 0:
        padshape = list(arr.shape)
        padshape[dim] = right
        arr = torch.cat((arr, torch.full(padshape, np.nan)).to(arr.device), dim=dim)
    return arr

In [ ]:
# split with nan
def split_with_nan(x, sections, axis=0):
    assert x.dtype in [np.float16, np.float32, np.float64]
    arrs = np.array_split(x, sections, axis=axis)
    target_length = arrs[0].shape[axis]
    for i in range(len(arrs)):
        arrs[i] = pad_nan_to_target(arrs[i], target_length, axis=axis)
    return arrs

In [ ]:
# pad with nan
def pad_nan_to_target(array, target_length, axis=0):
    assert array.dtype in [np.float16, np.float32, np.float64]
    pad_size = target_length - array.shape[axis]
    if pad_size <= 0:
        return array
    npad = [(0, 0)] * array.ndim
    npad[axis] = (0, pad_size)

    return np.pad(array, pad_width=npad, mode='constant', constant_values=np.nan)

In [ ]:
def centerize_vary_length_series(x):
    prefix_zeros = np.argmax(~np.isnan(x).all(axis=-1), axis=1)
    suffix_zeros = np.argmax(~np.isnan(x[:, ::-1]).all(axis=-1), axis=1)
    offset = (prefix_zeros + suffix_zeros) // 2 - prefix_zeros
    rows, column_indices = np.ogrid[:x.shape[0], :x.shape[1]]
    offset[offset < 0] += x.shape[1]
    column_indices = column_indices - offset[:, np.newaxis]

    return x[rows, column_indices]

In [ ]:
# random seed functions
def seed_everything(seed):
    pl.seed_everything(seed, workers=True)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
# pickle helper
def pkl_save(name, var):
    os.makedirs(os.path.dirname(name), exist_ok=True)
    with open(name, 'wb') as f:
        pickle.dump(var, f)

def pkl_load(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

# Constants

In [ ]:
# logger
LOG_LEVEL = logging.DEBUG

# datasets name
ELECTRICITY = "electricity"
M5 = "M5"
ETTh1 = "ETTh1"
ETTh2 = "ETTh2"
ETTm1 = "ETTm1"
ETTm2 = "ETTm2"
WEATHER = "WTH"

# backbones

Pyraformer = "Pyraformer"
TCN = "TCN"
NLinear = "NLinear"
Linear = "Linear"


# models
CoST = "CoST"
BACKBONE = Linear
MODEL = CoST + '-' + BACKBONE
DATASET = ETTm2

# device

DEVICE = torch.device('cpu') if not torch.cuda.is_available() else torch.device('cuda')

#hyperparameters

LR = 1e-3 #1e-3 #0.04365158322401657

# Train
BATCH_SIZE = 16
EPOCHS = None
ITERS = None
L = 201
UNIVARIATE = True

# Eval
EVALUATE = True
MAX_TRAIN_LENGTH = 201
PADDING = MAX_TRAIN_LENGTH-1
ENCODE_BATCH_SIZE = 256


# training
TRAIN = True
DETERMINISTIC = True
LOAD_MODEL = False
RESUME_TRAINING = False
MEMORY_PROFILING = False
LOAD_ENCODE = False

# training techniques

GRADIENT_CLIPPING = None # None is default, 0.5 is by norm
FIND_LR = False

# wandb

SETTINGS_STRING = "univariate" if UNIVARIATE else "multivariate"
RUN_ID = "0z980736"
RESUME_RUN = False
RUN_ID = wandb.util.generate_id() if not RESUME_RUN else RUN_ID
print(f"current RUN_ID is {RUN_ID}")


#paths
MODEL_SETTINGS_FOLDER = "/univariate" if UNIVARIATE else "/multivariate"
ROOT_FOLDER = "/content/drive/MyDrive/Tesi/code"
MODEL_FOLDER = ROOT_FOLDER + "/models"
CHECKPOINT_FOLDER = ROOT_FOLDER + "/checkpoints" + "/" + MODEL + "/" + DATASET + MODEL_SETTINGS_FOLDER
# LOGS_FOLDER = ROOT_FOLDER + "/logs"
ENCODING_FOLDER = ROOT_FOLDER + "/encoding/" + MODEL + "/" + DATASET + MODEL_SETTINGS_FOLDER
FORECASTING_RESULT = ROOT_FOLDER + "/forecasting_result/" + MODEL + "/" + DATASET + MODEL_SETTINGS_FOLDER
datasets_path = {
    ELECTRICITY: ROOT_FOLDER + "/datasets/electricity",
    ETTh1: ROOT_FOLDER + "/datasets/ETT",
    ETTh2: ROOT_FOLDER + "/datasets/ETT",
    ETTm1: ROOT_FOLDER + "/datasets/ETT",
    ETTm2: ROOT_FOLDER + "/datasets/ETT"
}

datasets_name = {
    ELECTRICITY: "/LD2011_2014.txt"
}
datasets_processed_name = {
    ELECTRICITY: "/electricity.csv",
    ETTh1: "/ETTh1.csv",
    ETTh2: "/ETTh2.csv",
    ETTm1: "/ETTm1.csv",
    ETTm2: "/ETTm2.csv"
}

datasets_pred_lens = {
    ELECTRICITY: [24, 48, 168, 336, 720], # [24, 48, 168, 336, 720]
    ETTh1: [24, 48, 168, 336, 720],
    ETTh2: [24, 48, 168, 336, 720],
    WEATHER: [24, 48, 168, 336, 720],
    M5: [28],
    ETTm1: [24, 48, 96, 288, 672],
    ETTm2: [24, 48, 96, 288, 672]
}

datasets_n_iters = {
    ELECTRICITY: 600,
    ETTh1: 200,
    ETTh2: 200,
    ETTm1: 200,
    ETTm2: 200
}

ITERS = datasets_n_iters[DATASET] if ITERS is None and EPOCHS is None else ITERS

config = dict(
    iters = ITERS,
    epochs = 0 if EPOCHS is None else EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate = LR,
    backbone = BACKBONE,
    dataset=DATASET,
    architecture=MODEL,
    run_id = RUN_ID)

current RUN_ID is 6l9p34oc


# WanDB

In [ ]:
# start a new experiment
run = wandb.init(project=config["architecture"] + "-" +config["dataset"], config = config, id = config["run_id"], resume = 'allow')
ENCODING_FOLDER += "/" + run.name
FORECASTING_RESULT += "/" + run.name
CHECKPOINT_FOLDER += "/" + run.name

# Logger

In [ ]:
# create logger
log = logging.getLogger('APP')
log.setLevel(LOG_LEVEL)
logging.basicConfig(level=LOG_LEVEL)

# # create console handler and set level to debug
# ch = logging.StreamHandler()
# ch.setLevel(logging.INFO)

# # create formatter
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# # add formatter to ch
# ch.setFormatter(formatter)

# # add ch to logger
# logger.addHandler(ch)

# Dataset and Datamodule

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, sigma, eval_mode = False,p = 0.5, multiplier = 10):
        super().__init__()
        self.data = data
        self.sigma = sigma
        self.eval_mode = eval_mode
        self.p = p
        self.multiplier = multiplier
        self.N, self.T, self.D = data.shape # num_ts, time, dim

    def __len__(self):
        # return self.data.shape[0] // self.look_window
        return self.data.shape[0] * self.multiplier

    def __getitem__(self, idx):
        ts = self.data[idx % self.N]

        return self.transform(ts), self.transform(ts)

    def get_len(self):
        return self.__len__()

    # def get_channels(self):
    #     return self.data.iloc[0, 1:].astype(str).str.replace(',', '.').astype('float32').shape[0]

    def transform(self, x):
        return self.jitter(self.shift(self.scale(x)))

    def jitter(self, x):
        if random.random() > self.p:
            return x
        return x + (torch.empty(x.shape).normal_(mean = 0, std = 0.5) * self.sigma)

    def scale(self, x):
        if random.random() > self.p:
            return x
        return x * (torch.empty(x.size(-1)).normal_(mean = 0, std = 0.5) * self.sigma + 1)

    def shift(self, x):
        if random.random() > self.p:
            return x
        return x + (torch.empty(x.size(-1)).normal_(mean = 0, std = 0.5) * self.sigma)

class CustomDataModule(pl.LightningDataModule):
    def __init__(self, name, path, batch_size, max_train_length,
                 encode_batch_size = 256, train_size = 0.6, test_size = 0.2, univariate = False):
        super().__init__()
        self.path = path # path to csv file
        self.batch_size = batch_size
        self.encode_batch_size = encode_batch_size
        self.max_train_length = max_train_length
        self.padding = max_train_length-1
        self.train_size = train_size
        self.test_size = test_size
        self.univariate = univariate
        self.name = name
        # self.data = np.load(path)

    # def prepare_data(self):
    #     # download

    def _get_time_features(self,dt):
        return np.stack([
                dt.minute.to_numpy(),
                dt.hour.to_numpy(),
                dt.dayofweek.to_numpy(),
                dt.day.to_numpy(),
                dt.dayofyear.to_numpy(),
                dt.month.to_numpy(),
                dt.weekofyear.to_numpy(),
            ], axis=1).astype(float)

    def _load_forecast_csv(self, path, name):
        data = pd.read_csv(path, index_col='date', parse_dates=True)
        dt_embed = self._get_time_features(data.index)
        n_covariate_cols = dt_embed.shape[-1]
        if self.univariate:
            if name in ('ETTh1', 'ETTh2', 'ETTm1', 'ETTm2'):
                data = data[['OT']]
            elif name == 'electricity':
                data = data[['MT_001']]
            elif name == 'WTH':
                data = data[['WetBulbCelsius']]
            else:
                data = data.iloc[:, -1:]
        data = data.to_numpy()
        # compute slices
        if name == 'ETTh1' or name == 'ETTh2':
            self.train_slice = slice(None, 12 * 30 * 24)
            self.valid_slice = slice(12 * 30 * 24, 16 * 30 * 24)
            self.test_slice = slice(16 * 30 * 24, 20 * 30 * 24)
        elif name == 'ETTm1' or name == 'ETTm2':
            self.train_slice = slice(None, 12 * 30 * 24 * 4)
            self.valid_slice = slice(12 * 30 * 24 * 4, 16 * 30 * 24 * 4)
            self.test_slice = slice(16 * 30 * 24 * 4, 20 * 30 * 24 * 4)
        elif name.startswith('M5'):
            self.train_slice = slice(None, int(0.8 * (1913 + 28)))
            self.valid_slice = slice(int(0.8 * (1913 + 28)), 1913 + 28)
            self.test_slice = slice(1913 + 28 - 1, 1913 + 2 * 28)
        else:
            self.train_slice = slice(None, int(self.train_size * len(data)))
            self.valid_slice = slice(int(self.train_size * len(data)), - int(self.test_size * len(data)))
            self.test_slice = slice(- int(self.test_size * len(data)), None)

        return data, dt_embed, n_covariate_cols

    def _scale_and_transform(self, data, dt_embed, n_covariate_cols, name):
        # scale data
        scaler = StandardScaler().fit(data[self.train_slice])
        data = scaler.transform(data)
        # NUM_ROWS x NUM_FEATURES -> NUM_FEATURES x NUM_ROWS x 1
        if name in ('electricity') or name.startswith('M5'):
            data = np.expand_dims(data.T, -1)  # Each variable is an instance rather than a feature
        else:
            data = np.expand_dims(data, 0)
        if n_covariate_cols > 0:
            dt_scaler = StandardScaler().fit(dt_embed[self.train_slice])
            dt_embed = np.expand_dims(dt_scaler.transform(dt_embed), 0)
            data = np.concatenate([np.repeat(dt_embed, data.shape[0], axis=0), data], axis=-1)

        return data, scaler, n_covariate_cols

    def _fit_setup(self, train_data):
        if self.max_train_length is not None:
            sections = train_data.shape[1] // self.max_train_length
        if sections >= 2:
            train_data = np.concatenate(split_with_nan(train_data, sections, axis=1), axis=0)

        temporal_missing = np.isnan(train_data).all(axis=-1).any(axis=0)
        if temporal_missing[0] or temporal_missing[-1]:
            train_data = centerize_vary_length_series(train_data)

        train_data = train_data[~np.isnan(train_data).all(axis=2).all(axis=1)]
        # pkl_save("/content/drive/MyDrive/Tesi/code/test/my_train_data.pkl", train_data)

        multiplier = 1 if train_data.shape[0] >= self.batch_size else math.ceil(self.batch_size / train_data.shape[0])

        return train_data, multiplier


    def setup(self, stage: str):
        # load csv
        data, dt_embed, n_covariate_cols = self._load_forecast_csv(self.path, self.name)

        # scale and transform
        self.data, self.scaler, self.n_covariate_cols = self._scale_and_transform(data, dt_embed, n_covariate_cols, self.name)

        # Assign train/val datasets for use in dataloaders
        if stage == "fit":
            train_data = self.data[:, self.train_slice]
            train_data, multiplier = self._fit_setup(train_data)
            # fit setup
            self.train = CustomDataset(torch.from_numpy(train_data).to(torch.float), sigma = 0.5, multiplier = multiplier)
            # self.validate = ElectricityDataset(data[self.valid_slice], self.look_window)
        if stage == 'encoding':
            self.encode = TensorDataset(torch.from_numpy(self.data).to(torch.float))




        # Assign test dataset for use in dataloader(s)
        # if stage == "test":
        #     self.test = ElectricityDataset(data[self.test_slice], self.look_window)

        # if stage == "predict":

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=min(self.batch_size, len(self.train)), drop_last = True, shuffle = True)

    def encode_dataloader(self):
        return DataLoader(self.encode, batch_size=self.encode_batch_size)

    # def test_dataloader(self):
    #     return DataLoader(self.test, batch_size=self.batch_size, drop_last = True)

    # def predict_dataloader(self):

## Preprocessing

### Electricity

In [ ]:
# df = pd.read_csv(datasets_path[ELECTRICITY] + datasets_name[ELECTRICITY], sep = ';')
# df.rename(columns={df.columns[0]: 'Date'},inplace=True)
# values = df.values
# values = values[:, 1:].astype(str)
# for i, value in enumerate(values):
#   values[i] = np.char.replace(value, ",", ".")
# values = values.astype(np.float32)
# np.save(datasets_path[ELECTRICITY] + "/electricity", values)

In [ ]:
# # # call this function to initialize the csv file
# def electricity_preprocess(path):
#     data_ecl = pd.read_csv(path + '/LD2011_2014.txt', parse_dates=True, sep=';', decimal=',', index_col=0)
#     data_ecl = data_ecl.resample('1h', closed='right').sum()
#     data_ecl = data_ecl.loc[:, data_ecl.cumsum(axis=0).iloc[8920] != 0]  # filter out instances with missing values
#     data_ecl.index = data_ecl.index.rename('date')
#     data_ecl = data_ecl['2012':]
#     data_ecl.to_csv(path + '/electricity.csv')
#     log.info("electriciy.csv created!")

In [ ]:
# electricity_preprocess(datasets_path[ELECTRICITY])

# Models

## OFFCIAL IMPLEMENTATIONS 🔽

## NLinear

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class NLinear(nn.Module):
    """
    Normalization-Linear
    """
    def __init__(self, covariate_size, d_model, layers, individual = True):
        super(NLinear, self).__init__()
        self.seq_len = covariate_size
        self.d_model = d_model

        # Use this line if you want to visualize the weights
        # self.Linear.weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
        self.channels = layers
        self.individual = individual
        if self.individual:
            self.Linear = nn.ModuleList()
            for i in range(self.channels):
                self.Linear.append(nn.Linear(self.seq_len,self.d_model))
        else:
            self.Linear = nn.Linear(self.seq_len, self.d_model)

    def forward(self, x):
        # x: [Batch, Input length, Channel]
        seq_last = x[:,-1:,:].detach()
        x = x - seq_last
        if self.individual:
            output = torch.zeros([x.size(0),self.d_model,x.size(2)],dtype=x.dtype).to(x.device)
            for i in range(self.channels):
                output[:,:,i] = self.Linear[i](x[:,:,i])
            x = output
        else:
            x = self.Linear(x.permute(0,2,1)).permute(0,2,1)
        x = x + seq_last
        return x # [Batch, Output length, Channel]

In [ ]:
x = torch.rand((16, 8, 201))
model = NLinear(8, 320, 4)
x = model(x)
x.shape

torch.Size([16, 320, 201])

## My Model 🔽

# Train

In [ ]:
# train function
def train(batch_size, datamodule, model, model_name, max_epochs = None, max_steps = -1, checkpoint_every_n_epochs = 50,
          check_val_every_n_epoch = 5, gradient_clip_val=0, resume_training = True, load_model = False,
          enable_checkpoint = True, monitor_metric = "val_loss", checkpoint_dir = None,
          log_flag = False, logs_dir = None,
          early_stopping = True, deterministic = False, profiler = None, find_lr = False):

    # check monitor metric
    assert monitor_metric in ["train_loss", "vall_loss"], "metric to monitor is invalid"

    # initialize callbacks array
    callbacks = [TQDMProgressBar(refresh_rate=20)]

    # add checkpoints to callbacks
    checkpoint_callback = None
    if enable_checkpoint and checkpoint_dir is not None:
        checkpoint_callback = ModelCheckpoint(dirpath=checkpoint_dir,  monitor = monitor_metric, filename=model_name + '{epoch:02d}-{' + monitor_metric + ':.2f}',
                                         save_last =True, every_n_epochs = checkpoint_every_n_epochs, save_on_train_epoch_end = True)
        callbacks.append(checkpoint_callback)

    # add early stopping to the callbacks
    if early_stopping:
        callbacks.append(EarlyStopping(monitor="val_loss", min_delta = 0.1, patience = 3, mode="min", check_on_train_epoch_end = False))

    # define the logger object
    logger = None
    if log_flag:
        # logger = TensorBoardLogger(logs_dir, name=model_name)
        wandb_logger = WandbLogger(name = model_name, log_model = 'all')

    # create the Trainer
    trainer = pl.Trainer(enable_checkpointing=enable_checkpoint, devices=1, accelerator="auto",
                         max_epochs=max_epochs, max_steps=max_steps, logger=logger, callbacks=callbacks,  ## remove max_step
                         check_val_every_n_epoch = check_val_every_n_epoch, gradient_clip_val=gradient_clip_val,## remove
                         deterministic = deterministic, profiler = profiler)

    if find_lr:
        model.train_dataloader
        tuner = Tuner(trainer)
        # Run learning rate finder
        lr_finder = tuner.lr_find(model, datamodule = datamodule, early_stop_threshold=None)

        # Plot with
        fig = lr_finder.plot(suggest=True)
        fig.show()
        # Pick point based on plot, or get suggestion
        new_lr = lr_finder.suggestion()
        log.info(f"the suggested lr is {new_lr}")
        model.lr = new_lr

    ckpt_path = None
    if resume_training:
        ckpt_path = checkpoint_dir + "/last.ckpt"
    trainer.fit(ckpt_path = ckpt_path, model=model, datamodule=datamodule)
    if checkpoint_callback is not None:
        log.info(checkpoint_callback.best_model_path)

    return trainer

#  Encoding

In [ ]:
def prepare_encoding(datamodule):
    datamodule.setup("encoding")
    loader = datamodule.encode_dataloader()
    return loader, datamodule.data.shape

def encode(model, data_shape, loader, batch_size, device, save_path):
    model.to(device)
    model = model.encoder_q
    res = model.encode(data_shape, loader, batch_size = batch_size, padding = PADDING)
    file_name = f"encoding_{time.time()}.pkl"
    pkl_save(f'{save_path}/{file_name}', res)
    pkl_save(f'{save_path}/last.pkl', res)
    log.info(f"encoding {file_name} saved")

    return res

# Forecasting Evaluation

In [ ]:
# Helper functions
def generate_pred_samples(features, data, pred_len, drop=0):
    n = data.shape[1]
    features = features[:, :-pred_len]
    labels = np.stack([ data[:, i:1+n+i-pred_len] for i in range(pred_len)], axis=2)[:, 1:]
    features = features[:, drop:]
    labels = labels[:, drop:]
    return features.reshape(-1, features.shape[-1]), labels.reshape(-1, labels.shape[2]*labels.shape[3])

def fit_ridge(train_features, train_y, valid_features, valid_y, MAX_SAMPLES=100000):
    # If the training set is too large, subsample MAX_SAMPLES examples
    if train_features.shape[0] > MAX_SAMPLES:
        split = train_test_split(
            train_features, train_y,
            train_size=MAX_SAMPLES, random_state=0
        )
        train_features = split[0]
        train_y = split[2]
    if valid_features.shape[0] > MAX_SAMPLES:
        split = train_test_split(
            valid_features, valid_y,
            train_size=MAX_SAMPLES, random_state=0
        )
        valid_features = split[0]
        valid_y = split[2]
    alphas = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]
    valid_results = []
    for alpha in alphas:
        # ipdb.set_trace(context=6)
        # log.debug(f"alpha: {alpha}")
        lr = Ridge(alpha=alpha).fit(train_features, train_y)
        valid_pred = lr.predict(valid_features)
        score = np.sqrt(((valid_pred - valid_y) ** 2).mean()) + np.abs(valid_pred - valid_y).mean()
        valid_results.append(score)
    best_alpha = alphas[np.argmin(valid_results)]
    log.info(f"best alpha: {best_alpha}")

    lr = Ridge(alpha=best_alpha)
    lr.fit(train_features, train_y)
    return lr

def cal_metrics(pred, target):
    return {
        'MSE': ((pred - target) ** 2).mean(),
        'MAE': np.abs(pred - target).mean()
    }

In [ ]:
def eval_forecasting(repr, data, train_slice, valid_slice, test_slice, n_covariate_cols, scaler, padding, pred_lens):
    train_repr = repr[:, train_slice]
    valid_repr = repr[:, valid_slice]
    test_repr = repr[:, test_slice]

    train_data = data[:, train_slice, n_covariate_cols:]
    valid_data = data[:, valid_slice, n_covariate_cols:]
    test_data = data[:, test_slice, n_covariate_cols:]

    ours_result = {}
    out_log = {}
    y_labels_mse = []
    y_labels_mae = []
    for pred_len in tqdm(pred_lens, desc=f"forecasting evaluation {pred_lens}"):
        train_features, train_labels = generate_pred_samples(train_repr, train_data, pred_len, drop=padding)
        valid_features, valid_labels = generate_pred_samples(valid_repr, valid_data, pred_len)
        test_features, test_labels = generate_pred_samples(test_repr, test_data, pred_len)

        lr = fit_ridge(train_features, train_labels, valid_features, valid_labels)

        test_pred = lr.predict(test_features)

        ori_shape = test_data.shape[0], -1, pred_len, test_data.shape[2]
        test_pred = test_pred.reshape(ori_shape)
        test_labels = test_labels.reshape(ori_shape)

        test_shape = test_pred.shape
        test_shape_swap = (test_shape[3], test_shape[1], test_shape[2], test_shape[0])
        if test_data.shape[0] > 1:
            test_pred_inv = scaler.inverse_transform(test_pred.swapaxes(0, 3)
                .reshape(-1, test_shape[0])).reshape(test_shape_swap).swapaxes(0, 3)
            test_labels_inv = scaler.inverse_transform(test_labels.swapaxes(0, 3)
                .reshape(-1, test_shape[0])).reshape(test_shape_swap).swapaxes(0, 3)
        else:
            test_pred_inv = scaler.inverse_transform(test_pred.reshape(-1, test_shape[3])).reshape(test_shape)
            test_labels_inv = scaler.inverse_transform(test_labels.reshape(-1, test_shape[3])).reshape(test_shape)

        # out_log[pred_len] = {
        #     # 'norm': test_pred,
        #     # 'raw': test_pred_inv
        #     # 'norm_gt': test_labels,
        #     # 'raw_gt': test_labels_inv
        # }
        ours_result[pred_len] = {
            'norm': cal_metrics(test_pred, test_labels),
            'raw': cal_metrics(test_pred_inv, test_labels_inv)
        }
        log.info(ours_result[pred_len])
        y_labels_mse.append(ours_result[pred_len]['norm']['MSE'])
        y_labels_mae.append(ours_result[pred_len]['norm']['MAE'])


    eval_res = {
        'ours': ours_result
    }
    return out_log, eval_res, y_labels_mse, y_labels_mae

# Main

In [ ]:
seeds = [1,2,3]
results = []

for seed in seeds:
    # initialize dataset
    # datamodule = ElectricityDataModule(datasets_path[ELECTRICITY] + datasets_processed_name[ELECTRICITY], BATCH_SIZE, L)
    datamodule = CustomDataModule(DATASET, datasets_path[DATASET] + datasets_processed_name[DATASET],
                        BATCH_SIZE, L, encode_batch_size=ENCODE_BATCH_SIZE, univariate=UNIVARIATE)

    # set seed if deterministic
    if DETERMINISTIC:
        seed_everything(seed)
    # initialize model, or load an extisting one
    if LOAD_MODEL:
        log.info("loading model...")
        model = CoSpyModel.load_from_checkpoint(CHECKPOINT_FOLDER + "/last.ckpt")
    else:
        net = OfficialCoSTEncoder(
                input_dims=8, output_dims=320,
                kernels=[1, 2, 4, 8, 16, 32, 64, 128],
                length=201,
                hidden_dims=64, depth=4,
                backbone = BACKBONE
            )
        model = OfficialCoSTModel(
            net,
            copy.deepcopy(net),
            kernels=[1, 2, 4, 8, 16, 32, 64, 128],
            dim=net.component_dims,
            alpha=0.0005,
            K=256,
            max_train_length = 201,
            epochs = EPOCHS,
            n_iters = ITERS,
            lr = LR
        )#CoSpyModel(L, epochs = EPOCHS, device = DEVICE, n_iters = ITERS, enc = ENCODER, lr = LR)

    if MEMORY_PROFILING:
        profiler = PyTorchProfiler()
    if TRAIN:
        trainer = train(BATCH_SIZE, datamodule, model, MODEL, max_epochs = EPOCHS, max_steps = ITERS, check_val_every_n_epoch = None, load_model = LOAD_MODEL,
                        gradient_clip_val=GRADIENT_CLIPPING,
            resume_training = RESUME_TRAINING, monitor_metric = "train_loss", checkpoint_dir = CHECKPOINT_FOLDER,
            early_stopping = False, deterministic = DETERMINISTIC, find_lr = FIND_LR)
    if EVALUATE:
        encoding_loader, data_shape = prepare_encoding(datamodule)
        if LOAD_ENCODE:
            log.info("load encoding...")
            repr = pkl_load(ENCODING_FOLDER + "/last.pkl")
        else:
            repr = encode(model, data_shape, encoding_loader, ENCODE_BATCH_SIZE, DEVICE, ENCODING_FOLDER)
    if EVALUATE:
        train_slice = datamodule.train_slice
        valid_slice = datamodule.valid_slice
        test_slice = datamodule.test_slice
        data = datamodule.data
        n_covariate_cols = datamodule.n_covariate_cols
        scaler = datamodule.scaler
        padding = PADDING
        pred_lens = datasets_pred_lens[DATASET]

        out, eval_res, y_labels_mse, y_labels_mae = eval_forecasting(repr, data, train_slice, valid_slice, test_slice,
                                        n_covariate_cols, scaler, padding, pred_lens)

        results.append([y_labels_mse, y_labels_mae])

        # data = [[x, y] for (x, y) in zip(pred_lens, y_labels_mse)]
        # table = wandb.Table(data=data, columns = ["pred_lens", "mse"])
        # wandb.log(
        # {f"{SETTINGS_STRING} forecasting MSE plot" : wandb.plot.scatter(table, "pred_lens", "mse",
        #     title=f"{SETTINGS_STRING} forecasting MSE norm plot")})

        # data = [[x, y] for (x, y) in zip(pred_lens, y_labels_mae)]
        # table = wandb.Table(data=data, columns = ["pred_lens", "mae"])
        # wandb.log(
        # {f"{SETTINGS_STRING} forecasting MAE plot" : wandb.plot.scatter(table, "pred_lens", "mae",
        #     title=f"{SETTINGS_STRING} forecasting MAE norm plot")})
        # wandb.log({"eval_forecasting": eval_res})
        pkl_save(FORECASTING_RESULT + "/out.pkl", out)
        pkl_save(FORECASTING_RESULT + "/eval_res.pkl", eval_res)

INFO:lightning_fabric.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
<ipython-input-16-19f5bcc49657>:70: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  dt.weekofyear.to_numpy(),
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                | Params
--------------------------------------------------
0 | encoder_q | OfficialCoSTEncoder | 18.6 M
1 | encoder_k | Off

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=200` reached.
INFO:APP:
<ipython-input-16-19f5bcc49657>:70: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  dt.weekofyear.to_numpy(),


data encoding:   0%|          | 0/1 [00:00<?, ?it/s]

sequence encoding:   0%|          | 0/69680 [00:00<?, ?it/s]

...

data encoding:   0%|          | 0/1 [00:00<?, ?it/s]

sequence encoding:   0%|          | 0/69680 [00:00<?, ?it/s]

INFO:APP:encoding encoding_1694179265.1157136.pkl saved


forecasting evaluation [24, 48, 96, 288, 672]:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:APP:best alpha: 1
INFO:APP:{'norm': {'MSE': 0.026827517586541883, 'MAE': 0.11584275080081889}, 'raw': {'MSE': 3.597937087046075, 'MAE': 1.3415454821717991}}
INFO:APP:best alpha: 1
INFO:APP:{'norm': {'MSE': 0.04830661066933399, 'MAE': 0.15962644467576131}, 'raw': {'MSE': 6.478577273477769, 'MAE': 1.8485933204476712}}
INFO:APP:best alpha: 1
INFO:APP:{'norm': {'MSE': 0.06622523640295494, 'MAE': 0.19060890023594615}, 'raw': {'MSE': 8.881710106215715, 'MAE': 2.2073932717644054}}
INFO:APP:best alpha: 1
INFO:APP:{'norm': {'MSE': 0.11846106652904725, 'MAE': 0.26053982296321804}, 'raw': {'MSE': 15.887249435213716, 'MAE': 3.017245527388102}}
INFO:APP:best alpha: 1
INFO:APP:{'norm': {'MSE': 0.16529641777234913, 'MAE': 0.31032646476996284}, 'raw': {'MSE': 22.168510650089942, 'MAE': 3.5938119797665147}}


In [ ]:
mse = []
mae = []
for i, pred_len in enumerate(pred_lens):
    mse.append(0)
    mae.append(0)
    for res in results:
        mse[i] += res[0][i] / len(results)
        mae[i] += res[1][i] / len(results)

log.info(mse)
log.info(mae)

data = [[x, y] for (x, y) in zip(pred_lens, mse)]
table = wandb.Table(data=data, columns = ["pred_lens", "mse"])
wandb.log(
{f"{SETTINGS_STRING} forecasting MSE plot" : wandb.plot.scatter(table, "pred_lens", "mse",
    title=f"{SETTINGS_STRING} forecasting MSE norm plot")})

data = [[x, y] for (x, y) in zip(pred_lens, mae)]
table = wandb.Table(data=data, columns = ["pred_lens", "mae"])
wandb.log(
{f"{SETTINGS_STRING} forecasting MAE plot" : wandb.plot.scatter(table, "pred_lens", "mae",
    title=f"{SETTINGS_STRING} forecasting MAE norm plot")})


INFO:APP:[0.02767434649844249, 0.04918789892038547, 0.06709897723587861, 0.11782806062256995, 0.1639527146436185]
INFO:APP:[0.1182246483926824, 0.1616641315562232, 0.19234729375800472, 0.26075920336493075, 0.309325898105969]
